In [2]:
%cd ..

/Users/mateoibarguen/Desktop/DATA-Capstone


In [66]:
from spectra_models import *
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, LSTM, TimeDistributed, ConvLSTM2D, BatchNormalization, MaxPooling1D, Conv1D, Bidirectional
import pandas as pd
import pickle

## Load data

In [4]:
train_path = 'spectra_models/data/set_01/train_01.pkl'
test_path = 'spectra_models/data/set_01/test_01.pkl'
spectra_preprocessor = SpectraPreprocessor(train_filename=train_path, test_filename=test_path)

In [5]:
X_train, y_train, X_test, y_test = spectra_preprocessor.transform(encoded=True)

## Model Experimentation

In [6]:
baseline_model_keras = Sequential()
baseline_model_keras.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(1001, 10, 1)))
baseline_model_keras.add(Conv2D(16, kernel_size=3, activation='relu'))
baseline_model_keras.add(Flatten())
baseline_model_keras.add(Dense(8, activation='relu'))
baseline_model_keras.add(Dense(5, activation='sigmoid'))

baseline_model_compile_dict = {'optimizer': 'adam', 'loss': 'categorical_crossentropy', 'metrics':['accuracy']}

In [7]:
baseline_model = BaseModel(baseline_model_keras)

In [8]:
baseline_model.fit(X_train, y_train, X_test, y_test, batch_size=16, epochs=1, compile_dict=baseline_model_compile_dict)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 6000 samples, validate on 1500 samples
Epoch 1/1
2500/2500 [==============================] - 2s 725us/step


In [13]:
save_model(model_name='01_cnn_baseline_model', model= baseline_model, train_path=train_path, test_path=test_path)

Saved model results as 01_cnn_baseline_model in the file: spectra_models/model_results/01_cnn_baseline_model


In [70]:
def save_model(model_name, train_path, test_path, model):
    model_configs = get_model_params(model_name, train_path, test_path, model)
    with open(MODEL_RESULTS_PATH, 'w') as f:
        json.dump(str(model_configs), f)
        
    print(f'Saved model results as {model_name} in the file: {MODEL_RESULTS_PATH}')

In [60]:
import json

In [72]:
with open('spectra_models/model_results.txt', 'r') as f:
    a = json.load(f)

### Research counting nn. 

### LSTM

In [43]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

In [48]:
X_train, y_train, X_test, y_test = spectra_preprocessor.transform(encoded=True)

In [62]:
# define LSTM
cnn = Sequential()
cnn.add(Conv1D(64, kernel_size=3, activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())

model = Sequential()
model.add(TimeDistributed(cnn))
model.add(LSTM(100))
model.add(Dense(5, activation='softmax'))

In [63]:
lstm_cnn = BaseModel(model)

In [65]:
#lstm_cnn.fit(X_train, y_train, X_test, y_test, batch_size=16, epochs=3, compile_dict=baseline_model_compile_dict)

In [26]:
from keras import Model

## BiLSTM

In [111]:
X_train, y_train, X_test, y_test = spectra_preprocessor.transform(encoded=True)

In [116]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

In [117]:
bilstm = Sequential()
n_timesteps = 1001
bilstm.add(Bidirectional(LSTM(64), input_shape=(n_timesteps, 10)))
bilstm.add(Dense(5, activation='softmax'))

In [118]:
bilstm_model = BaseModel(bilstm)

In [120]:
# bilstm_model.fit(X_train, y_train, X_test, y_test, batch_size=16, epochs=1, compile_dict=baseline_model_compile_dict)

save_model('a', 'train.pkl', 'test.pkl', baseline_model)

- Try **ensemble** of different channels. 
- Baseline: logistic regression or SVM. 
    - We want to know whether or not NN's are doing anything. 
    

In [130]:
from keras.layers import Layer

In [121]:
from keras.layers import BatchNormalization, CuDNNGRU

In [127]:
from keras.layers import BatchNormalization, CuDNNGRU

model = Sequential()
model.add(BatchNormalization(momentum=0.98,input_shape=(1001, 10)))
model.add(Bidirectional(CuDNNGRU(128, return_sequences = True)))
model.add(Flatten())
model.add(Dense(5,activation='sigmoid'))
print(model.summary())

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 1001, 10)          40        
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 1001, 256)         107520    
_________________________________________________________________
flatten_18 (Flatten)         (None, 256256)            0         
_________________________________________________________________
dense_28 (Dense)             (None, 5)                 1281285   
Total params: 1,388,845
Trainable params: 1,388,825
Non-trainable params: 20
_________________________________________________________________
None


In [128]:
cudnn = BaseModel(model)

In [129]:
cudnn.fit(X_train, y_train, X_test, y_test, batch_size=16, epochs=1, compile_dict=baseline_model_compile_dict)

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node bidirectional_18/CudnnRNN (defined at /Applications/anaconda3/lib/python3.7/site-packages/keras/layers/cudnn_recurrent.py:297) with these attrs: [seed=87654321, dropout=0, input_mode="linear_input", T=DT_FLOAT, direction="unidirectional", rnn_mode="gru", seed2=0, is_training=true]
Registered devices: [CPU]
Registered kernels:
  <no registered kernels>

	 [[bidirectional_18/CudnnRNN]]

Errors may have originated from an input operation.
Input Source operations connected to node bidirectional_18/CudnnRNN:
 bidirectional_18/transpose (defined at /Applications/anaconda3/lib/python3.7/site-packages/keras/layers/cudnn_recurrent.py:271)	
 bidirectional_18/ExpandDims_1 (defined at /Applications/anaconda3/lib/python3.7/site-packages/keras/layers/cudnn_recurrent.py:273)	
 bidirectional_18/concat (defined at /Applications/anaconda3/lib/python3.7/site-packages/keras/layers/cudnn_recurrent.py:60)